# Set up plates for a duckweed genotypes x media growth assay
Note that you'll need to be connected to a Jubilee machine to run these cells

## 0. Import packages and libraries

In [ ]:
import os
from pathlib import Path
import json
import time
import ipywidgets as widgets
import pandas as pd

from science_jubilee.Machine import Machine
from science_jubilee.decks.Deck import Deck
from science_jubilee.tools.Syringe import Syringe
from science_jubilee.tools.Loop import Loop
from science_jubilee.tools.Camera import Camera
# import utils.DuckbotExptSetupUtils as exp

%load_ext autoreload
%autoreload 2

In [ ]:
# Initialize your connection to the machine
m = Machine()
deck = m.load_deck("lab_automation_deck_MA")

loop = Loop(0, "Loop")
camera = Camera(3, "Camera")
syringe = Syringe(4, "Syringe", "10cc_syringe")

m.load_tool(loop)
m.load_tool(camera)
m.load_tool(syringe)

## 1. Select config files

### Select experimental config file

In [ ]:
root_dir = Path(os.path.abspath('')).resolve().parent
config_file_dir = os.path.join(root_dir, "experiments")

config_opts = os.listdir(config_file_dir)
expt_choice = widgets.Dropdown(options = config_opts)
display(expt_choice)

In [ ]:
print(f"You selected: *{expt_choice.value}*")
print("If this is not correct re-select before continuing")

### Visualize and confirm plate set up

#### Run this cell to print a visualization of your experimental design. If this isn't as expected go back and re-run the config file selection cell or go back and directly edit your experimental config file and start again

In [ ]:
%matplotlib inline
from utils.GrowthAssayHelpers import visualize_plate_setup

experimental_config_file = os.path.join(config_file_dir, expt_choice.value, f'{expt_choice.value}.json')
visualize_plate_setup(experimental_config_file)

## 2. Label 24-well plates and add to machine
Follow the instructions that are printed when you run the cells

#### Label Plates and add plates to machine

In [ ]:
with open(experimental_config_file, 'r') as f:
    experiment_data = json.load(f)
plate_info = experiment_data['sample_info']
df = pd.DataFrame(plate_info)
num_plates = df.plate.nunique()

print("This experiment requires {} 24-well plate(s)".format(num_plates))
print("----")
lst = list(range(1,num_plates + 1))
for n in lst:
    print("Label a plate with experiment ID or initials and 'plate {}'".format(n))
print("----")
print ("Place the 24-well plate(s) in the jubilee".format(num_plates))
print ("Start at position 1 and fill empty plate slots in order")

In [ ]:
# Define the labware you will be using for this experiment.
# Slot positions should match the location you inserted each labware in the previous cell.
# Add the media reservoir; I'll swap out petri dishes to slot 0
reservoir = deck.load_labware("generic_petri_dish_100ml", 0)

# We'll use three 24-well plates.
plate1 = deck.load_labware("greiner_24_wellplate_3300ul", 1)
plate2 = deck.load_labware("greiner_24_wellplate_3300ul", 2)
plate3 = deck.load_labware("greiner_24_wellplate_3300ul", 3)

## 3. Dispense media
Run cells, making sure to follow the instructions that print when your run the cells

In [ ]:
# Pickup syringe
m.pickup_tool(syringe)

In [ ]:
from utils.GrowthAssayHelpers import dispense_media_to_wells

dispense_media_to_wells(m, syringe, df)

## 4. Transfer duckweed

### Transfer option 1: Automated transfer with inoculation loop

In [ ]:
#pop the bed down to avoid any collisions on tool change
m.move_to(z=50)

In [ ]:
# pick up the innoculation loop
m.pickup_tool(loop)

In [ ]:
# Inoculation Loop Transfer
# The machine will move after running this cell
from utils.GrowthAssayHelpers import inoculation_loop_transfer

inoculation_loop_transfer(m, loop, df)

In [ ]:
# pop bed down to access labware
m.move_to(z=100)

### Consecutive Tranfser Passes
These cells will find any wells missing duckweed, and then prompt you to swap in the appropriate duckweed reservoir for transfer. Re-run the cells as many times as you like until you are satisfied with the results

In [ ]:
m.pickup_tool(camera)

In [ ]:
%matplotlib inline
from utils.GrowthAssayHelpers import check_wells

check_wells(m, camera, df)

In [ ]:
from utils.GrowthAssayHelpers import fill_empty_wells

m.pickup_tool(loop)

fill_empty_wells(m, loop, df)

### Transfer option 2: Manual transfer
Run this cell to generate written instructions of what duckweed genotypes should be in which wells. 

In [ ]:
#Manual transfer

grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    for index,s in sample_df.iterrows():
        plate = s["plate"]
        well = s["well"]
        print(f"Transfer {field_value} to plate {plate}, well {well}")
    input()